In [1]:
import requests
from bs4 import BeautifulSoup
import html5lib
import pandas as pd
from unidecode import unidecode
import random
import langid
import pickle

### 1. Obtendo lista de estilos

In [ ]:
url = 'https://www.cifraclub.com/estilos'

page = requests.get(url)
html_doc = page.text
soup = BeautifulSoup(html_doc, 'html5lib')

lista_estilos = []
for i in soup.find(class_='g-1 list-links est-all g-sb').find_all('a'):
    lista_estilos.append(i.get('href').replace('/',''))

In [ ]:
lista_estilos[:5]

In [ ]:
with open('lista_estilos.pkl', 'wb') as f:
    pickle.dump(lista_estilos, f)

### 2. Obtendo lista de top artistas por estilo

In [ ]:
dict_artistas_por_estilo = {}

for estilo in lista_estilos:
    url = 'https://www.cifraclub.com.br/mais-acessadas/' + estilo

    page = requests.get(url)
    html_doc = page.text
    soup = BeautifulSoup(html_doc, 'html5lib')

    lista_artistas_estilo = []
    for i in soup.find(class_="top top__big thumb_big top_art topArtist topArtist--hiddenColumn").find_all('a'):
        lista_artistas_estilo.append(i.get('href').replace('/', ''))
    dict_artistas_por_estilo[estilo] = lista_artistas_estilo

    print(estilo)

In [ ]:
dict_artistas_por_estilo

In [ ]:
# Complemento necessário: aumentar a base para incluir artistas nacionais em estilos onde há muitos gringos, 
# corrigir estilo de alguns artistas, agrupar estilos

df_estilos = pd.Series(dict_artistas_por_estilo, name='artista').rename_axis('estilo').explode().reset_index()
df_estilos.to_excel('df_estilos.xlsx', index=False)




In [2]:
# Fiz os ajustes pontualmente direto na planilha
df_estilos = pd.read_excel('df_estilos_ajustes.xlsx')
df_estilos['artista'] = df_estilos['artista'].apply(lambda x: unidecode(x).lower().replace(' ', '-'))

df_estilos.head()

,estilo,artista
0,indie,red-hot-chili-peppers
1,indie,los-hermanos
2,indie,radiohead
3,indie,kamaitachi
4,indie,audioslave


In [3]:
df_estilos.to_pickle('df_estilos.pkl')

### 3. Obtendo lista de músicas por artista
Vou pegar somente as que possuem cifra

In [12]:
nova_lista_estilos = list(set(df_estilos['estilo']))

In [24]:
dict_musicas_por_artista = {}
    
for estilo in nova_lista_estilos:
    for artista in list(df_estilos[df_estilos['estilo'] == estilo]['artista']):
        url = 'https://www.cifraclub.com/' + artista

        page = requests.get(url)
        html_doc = page.text
        soup = BeautifulSoup(html_doc, 'html5lib')

        lista_musicas_por_artista = []
        if soup.find(class_='list-links art_musics alf all artistMusics--allSongs') is None:
            for i in soup.find(class_='list-links art_musics all listSongArtist').find_all(class_='art_music-link'):
                lista_musicas_por_artista.append([i.get('href'), i.get('href').split('/')[2]])
        else:
            for i in soup.find(class_='list-links art_musics alf all artistMusics--allSongs').find_all(class_='art_music-link'):
                lista_musicas_por_artista.append([i.get('href'), i.get('href').split('/')[2]])

        lista_musicas_por_artista = [k for k in lista_musicas_por_artista if 'letra' not in k[0]]
        dict_musicas_por_artista[artista] = lista_musicas_por_artista
        print(estilo, artista)

infantil disney
infantil dragon-ball-gt
infantil 3-palavrinhas
infantil galinha-pintadinha
infantil temas-infantis
infantil a-turma-do-balao-magico
infantil mundo-bita
infantil steven-universo
infantil viva-a-vida-uma-festa
infantil naruto
infantil hora-de-aventura
infantil o-rei-leao
infantil frozen
infantil xuxa
infantil chaves
infantil toy-story
infantil dragon-ball
infantil os-cavaleiros-do-zodiaco-musicas
infantil cantigas-populares
infantil trem-da-alegria
infantil moana-trilha-sonora
infantil palavra-cantada
infantil pequeno-cidadao
infantil violetta
infantil bob-esponja
infantil adventure-time
infantil soy-luna
infantil turma-do-printy
infantil carrossel-2012
infantil encanto-disney
infantil pinkfong
infantil dragonball-z
infantil para-nossa-alegria
infantil tarzan
infantil cantinho-da-crianca
infantil barbie
infantil aladdin
infantil eliana
infantil larissa-manoela
infantil a-turma-do-seu-lobato
infantil patati-patata
infantil a-bela-e-a-fera
infantil cancoes-escoteiras
infant

regional joan-sebastian
regional joao-luiz-correa
regional wilson-paim
regional caprichoso-boi-bumba
regional quarteto-coracao-de-potro
regional tche-barbaridade
regional intocable
regional gabriel-sater
regional banda-ms
regional christian-nodal
regional rolando-boldrin
regional jairo-lambari-fernandes
regional cesar-passarinho
regional leonel-gomes
regional gildo-de-freitas
regional noel-guarany
regional angela-aguilar
regional leopoldo-rassier
regional joca-martins
regional gaucho-da-fronteira
regional xangai
regional lisandro-amaral
regional mano-lima
regional quim-barreiros
regional folcloricas-gauchas
regional os-bertussi
regional grupo-frontera
regional pirisca-grecco
regional cristiano-quevedo
regional cenair-maica
regional calibre-50
regional ricardo-bergha
regional joao-chagas-leite
mpb capiba
mpb alceu-valenca
mpb djavan
mpb ze-ramalho
mpb caetano-veloso
mpb anavitoria
mpb nando-reis
mpb chico-buarque
mpb tribalistas
mpb marisa-monte
mpb fagner
mpb seu-jorge
mpb toquinho
mpb

indie brian-jonestown-massacre
indie tagore
indie som-imaginario
indie diiv
indie hawkwind
indie jon-anderson
indie love
indie temples
indie sly-and-the-family-stone
indie spiritualized
indie the-dream-academy
indie kula-shaker
indie 13th-floor-elevators
indie quicksilver-messenger-service
indie daniel-ash
indie dead-meadow
indie country-joe-and-the-fish
indie spencer-davis-group
indie supercordas
indie os-skywalkers
indie moby-grape
indie frans-bauer
indie uncle-acid-and-the-deadbeats
indie kaleidoscope
indie earth-and-fire
indie ervilhas-astrais
indie zola-jesus
indie the-seeds
indie a1000noacido
indie laranja-freak
indie uli-jon-roth
indie black-countess
indie os-haxixins
indie headmeat
indie mercado-de-peixe
indie dom-pepo
indie coven-13
indie gong
indie morrah
indie federico-aubele
indie gilberto-lourenco
gospelreligioso harpa-crista
gospelreligioso gabriela-rocha
gospelreligioso catolicas
gospelreligioso aline-barros
gospelreligioso fernandinho
gospelreligioso ministerio-morada
g

hip-hop-rap baco-exu-do-blues
hip-hop-rap hungria-hip-hop
hip-hop-rap pele-milflows
hip-hop-rap l7nnon
hip-hop-rap flora-matos
hip-hop-rap post-malone
hip-hop-rap projota
hip-hop-rap mc-xama
hip-hop-rap vmz
hip-hop-rap racionais-mcs
hip-hop-rap pollo
hip-hop-rap rael-da-rima
hip-hop-rap luiz-lins
hip-hop-rap sant
hip-hop-rap tribo-da-periferia
hip-hop-rap criolo
hip-hop-rap oriente
hip-hop-rap eminem
hip-hop-rap 3030
hip-hop-rap froid
hip-hop-rap filipe-ret
hip-hop-rap 7-minutoz
hip-hop-rap konai
hip-hop-rap xxxtentacion
hip-hop-rap emicida
hip-hop-rap dama
hip-hop-rap gabriel-pensador
hip-hop-rap yago-oproprio
hip-hop-rap alex-g
hip-hop-rap lil-peep
hip-hop-rap kanye-west
hip-hop-rap wos
hip-hop-rap the-sugarhill-gang
hip-hop-rap blakkstar
hip-hop-rap frank-ocean
hip-hop-rap akon
hip-hop-rap tyler-the-creator
hip-hop-rap wiz-khalifa
hip-hop-rap childish-gambino
hip-hop-rap sampa-crew
hip-hop-rap fabio-brazza
hip-hop-rap lil-nas-x
hip-hop-rap luckhaos
hip-hop-rap powfu
hip-hop-rap drak

In [25]:
len(dict_musicas_por_artista.keys())

1434

In [26]:
with open('dict_musicas_por_artista.pkl', 'wb') as f:
    pickle.dump(dict_musicas_por_artista, f)

In [30]:
# Identificando artistas nacionais

def detecta_portugues(artista):
    df = pd.DataFrame(dict_musicas_por_artista[artista])
    df.columns = ['link', 'mus']
    df['artista'] = artista
    df['mus'] = df['mus'].apply(lambda x: x.replace('-', ' '))
    df['lang'] = df['mus'].apply(lambda x: langid.classify(x)[0])
    if 'pt' in list(df[['lang', 'mus']].groupby(['lang']).count().sort_values('mus', ascending = False).head(3).index):
        flag = 1
    else:
        flag = 0
    return flag

lista_nacionais = []
count = 0
for artista in list(dict_musicas_por_artista.keys()):
    if len(dict_musicas_por_artista[artista])>0:
        if detecta_portugues(artista) == 1:
            lista_nacionais.append(artista)
        count+=1
        if count % 50 == 0:
            print(artista, count)

dict_musicas_por_artista_ajustado = {key: dict_musicas_por_artista[key] for key in lista_nacionais}


jesse 50
lulu-santos 100
bon-jovi 150
plebe-rude 200
pixote 250
major-rd 300
tayrone-cigano 350
pirisca-grecco 400
ney-matogrosso 450
nattan 500
cultura-profetica 550
ricardo-braga 600
tiago-nacarato 650
tim-bernardes 700
esteban 750
daniel-ash 800
padre-zezinho 850
casuarina 900
francisco-alves 950
no-use-for-a-name 1000
blind-guardian 1050
desree 1100
tyler-the-creator 1150
bruno-e-marrone 1200
mc-marcinho 1250
barto-galeno 1300
banda-beijo 1350


In [33]:
# Selecionando amostra de musicas para analise

count = 0
for artista in list(dict_musicas_por_artista_ajustado.keys()):

    lista = dict_musicas_por_artista[artista]
    if len(lista) >= 70:
        amostra = 70
    else:
        amostra = min(len(lista), 35)

    lista_amostra = random.sample(lista,amostra)
    dict_musicas_por_artista_ajustado[artista] = lista_amostra
    
    count+=1
    if count % 100 == 0:
        print(artista, count, len(dict_musicas_por_artista_ajustado[artista]))


os-paralamas-do-sucesso 100 70
gabriel-diniz 200 70
vander-lee 300 70
felipe-amorim 400 18
love 500 21
gavioes-da-fiel 600 8
sant 700 6
mc-daleste 800 10


In [34]:
len(list(dict_musicas_por_artista_ajustado.keys())) # qtde de artistas

873

In [35]:
sum([len(dict_musicas_por_artista_ajustado[k]) for k in list(dict_musicas_por_artista_ajustado.keys())]) # qtde de músicas


33004

In [36]:
with open('dict_musicas_por_artista_ajustado.pkl', 'wb') as f:
    pickle.dump(dict_musicas_por_artista_ajustado, f)

### 4. Obtendo cifras e letras

In [37]:
# Funções para coleta de cifra e letra

def pega_cifra(artista, musica):
    url = 'https://www.cifraclub.com/' + artista + '/' + musica
    page = requests.get(url)
    html_doc = page.text
    soup = BeautifulSoup(html_doc, 'html5lib')
    if soup.find('pre') == None:
        lista_acordes = 'sem_info'
        tonalidade = 'sem_info'
    else:
        lista_acordes = str(soup.find('pre').find_all('b'))
        if soup.find(id = 'cifra_tom').find('a') == None:
            tonalidade = 'sem_info'
        else:
            tonalidade = str(soup.find(id = 'cifra_tom').find('a').text)
    return (lista_acordes, tonalidade)

def pega_letra(artista, musica):
    url = 'https://www.cifraclub.com/' + artista + '/' + musica + '/letra'
    page = requests.get(url)
    html_doc = page.text
    soup = BeautifulSoup(html_doc, 'html5lib')
    if soup.find(class_ = 'letra') == None:
        lista_letra = 'sem_info'
    else:
        lista_letra = str(soup.find(class_ = 'letra').find_all('p'))
    return lista_letra

In [38]:
with open('dict_musicas_por_artista_ajustado.pkl', 'rb') as handle:
    dict_musicas_por_artista_ajustado = pickle.load(handle)

In [77]:
# Obtendo cifra e letra a partir do artista/música

#lista_dados_musica = []
count = 853

for artista in list(dict_musicas_por_artista_ajustado.keys())[853:]:
    for musica in dict_musicas_por_artista_ajustado[artista]:
        musica = musica[1]
        tonalidade = pega_cifra(artista, musica)[1]
        lista_cifra = pega_cifra(artista, musica)[0]
        lista_letra = pega_letra(artista, musica)
        lista_dados_musica.append([artista, musica, tonalidade, lista_cifra, lista_letra])
    print(artista, count)
    count += 1
    

banda-beijo 853
jauperi 854
harmonia-do-samba 855
so-brega 856
banda-mel 857
e-o-tchan 858
claudia-leitte 859
tomate 860
chicabana 861
os-tincoas 862
jose-cid 863
filhos-de-jorge 864
bell-marques 865
psirico 866
terrasamba 867
ceceu-muniz 868
banda-luxuria 869
parangole 870
talita-costa 871
ricardo-chaves 872


In [78]:
with open('lista_dados_musica_11.pkl', 'wb') as f:
    pickle.dump(lista_dados_musica, f)